Imports. The output of `tf.__version__` should read something like 2.#

In [1]:
import os
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
import pycocotools
import tensorflow as tf
from absl import flags
from pathlib import Path
print(tf.__version__)
import logging
logging.basicConfig(level=logging.INFO)
from tensorflow.python.client import device_lib
for d in device_lib.list_local_devices():
    print(d)
    print('---')

2.6.0
name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1296173031685791894

---
name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7769489408
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17937536419887331623
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:04:00.0, compute capability: 6.1"

---


2021-10-02 11:16:30.287061: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-02 11:16:30.503412: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1885] Ignoring visible gpu device (device: 1, name: NVS 510, pci bus id: 0000:03:00.0, compute capability: 3.0) with Cuda compute capability 3.0. The minimum required Cuda capability is 3.5.
2021-10-02 11:16:30.958285: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:0 with 7409 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:04:00.0, compute capability: 6.1


Set directories.

In [2]:
DATA_DIR = Path("./data/20k")
TRAIN_FILE = DATA_DIR / "20k_train.tfrecord"
TEST_FILE = DATA_DIR / "20k_test.tfrecord"
CLASS_LABELS_FILE = DATA_DIR / "classlabels.txt"
CKPT_DIR = Path("./checkpoints") / "ckpts07"
EXPORT_DIR = Path("./export/") / "07"
os.makedirs(CKPT_DIR, exist_ok=True)
os.makedirs(EXPORT_DIR, exist_ok=True)

print("Reading data from ", DATA_DIR.resolve())
print("Saving Checkpoints to ", CKPT_DIR.resolve())
print("Exporting to ", EXPORT_DIR.resolve())

Reading data from  /home/ms/prg/train_targetID/data/20k
Saving Checkpoints to  /home/ms/prg/train_targetID/checkpoints/ckpts07
Exporting to  /home/ms/prg/train_targetID/export/07


Read class labels.

In [3]:
with open(CLASS_LABELS_FILE, "r") as f:
    label_map = [s.strip() for s in f.readlines()]
    
label_map_dict = {}
for i, l in enumerate(label_map):
    label_map_dict[i+1] = l

for k, v in label_map_dict.items():
    print(k, ": ", v)

1 :  circle
2 :  diamond
3 :  halfcircle
4 :  heart
5 :  hexagon
6 :  hexstar
7 :  moon
8 :  octagon
9 :  pentagon
10 :  pentstar
11 :  plus
12 :  square
13 :  trapezoid
14 :  triangle


In [4]:
train_data = object_detector.DataLoader(
    tfrecord_file_patten = str(TRAIN_FILE.resolve()),
    size = 18000,
    label_map = label_map_dict
)

test_data = object_detector.DataLoader(
    tfrecord_file_patten = str(TEST_FILE.resolve()),
    size = 2000,
    label_map = label_map_dict
)

Create spec with hyperparameters.

In [5]:
hparams = {
    "map_freq" : 2,
    "learning_rate" : 0.2,
}
spec = object_detector.EfficientDetLite3Spec(
    hparams=hparams, model_dir=CKPT_DIR, epochs=300, batch_size=32,
    var_freeze_expr='(efficientnet|fpn_cells|resample_p6)',
    tflite_max_detections = 10,
    verbose = 1,
)

2021-10-02 11:16:31.016680: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1885] Ignoring visible gpu device (device: 1, name: NVS 510, pci bus id: 0000:03:00.0, compute capability: 3.0) with Cuda compute capability 3.0. The minimum required Cuda capability is 3.5.
2021-10-02 11:16:31.021080: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7409 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:04:00.0, compute capability: 6.1


Train model

In [ ]:
model = object_detector.create(
    train_data = train_data,
    model_spec = spec,
    validation_data = test_data,
    train_whole_model = False,
    do_train = True,
)
model.export(
    export_dir = EXPORT_DIR,
    export_format = [ExportFormat.TFLITE, ExportFormat.LABEL, ExportFormat.SAVED_MODEL]
)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...
INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:target_size = (512, 512), output_size = (512, 512)
INFO:absl:LR schedule method: cosine
INFO:absl:Use SGD optimizer
2021-10-02 11:16:49.619383: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/300


2021-10-02 11:17:11.259404: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8204


562/562 [==============================] - 699s 1s/step - det_loss: 0.9097 - cls_loss: 0.7376 - box_loss: 0.0034 - reg_l2_loss: 0.0090 - loss: 0.9187 - learning_rate: 0.0540 - gradient_norm: 0.8468 - val_det_loss: 0.8201 - val_cls_loss: 0.7192 - val_box_loss: 0.0020 - val_reg_l2_loss: 0.0102 - val_loss: 0.8303

Epoch 00001: saving model to checkpoints/ckpts07/ckpt-1
Epoch 2/300
562/562 [==============================] - 691s 1s/step - det_loss: 0.6490 - cls_loss: 0.5354 - box_loss: 0.0023 - reg_l2_loss: 0.0116 - loss: 0.6606 - learning_rate: 0.1000 - gradient_norm: 0.7164 - val_det_loss: 0.6173 - val_cls_loss: 0.5264 - val_box_loss: 0.0018 - val_reg_l2_loss: 0.0129 - val_loss: 0.6302

Epoch 00002: saving model to checkpoints/ckpts07/ckpt-2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got

INFO:absl:use max_reduce for pre-nms topk.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
creating index...
index created!
Loading and preparing results...
Converting ndarray to lists...
(198400, 7)
0/198400
DONE (t=1.77s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=10.48s).
Accumulating evaluation results...
DONE (t=0.09s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.116
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.185
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.118
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.009
 Average Pr

Epoch 10/300
562/562 [==============================] - 689s 1s/step - det_loss: 0.5060 - cls_loss: 0.4175 - box_loss: 0.0018 - reg_l2_loss: 0.0254 - loss: 0.5313 - learning_rate: 0.0998 - gradient_norm: 0.5551 - val_det_loss: 0.4676 - val_cls_loss: 0.3927 - val_box_loss: 0.0015 - val_reg_l2_loss: 0.0260 - val_loss: 0.4936

Epoch 00010: saving model to checkpoints/ckpts07/ckpt-10
creating index...
index created!
Loading and preparing results...
Converting ndarray to lists...
(198400, 7)
0/198400
DONE (t=1.72s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=10.48s).
Accumulating evaluation results...
DONE (t=0.08s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.211
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.267
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.243
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.006